In [3]:
# https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e
# General imports
import glob
import os
import re
import pickle
import datetime
import itertools

# Data manipulation
#import growth_analysis as ga
import pandas as pd
#from multiprocesspandas import applyparallel
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [2]:
# make list of all variants in 4 site landscape
AA_list = list('ACDEFGHIKLMNPQRSTVWY')

all_seq = list(itertools.product(AA_list, repeat=4))

all_seq = list(map(lambda sub: "".join(sub), all_seq))

In [4]:
# load measured protein data
measured = pd.read_csv('112023_tuned_realx3_mut_pred_synthetic_landscape_TrpB.csv')
# measured = measured[measured['# Stop'] == 0].copy()
# measured.drop(columns=['# Stop'], inplace=True)
measured['imputed'] = False

# # get missing sequences
# measured_seq = list(measured['AAs'])
# missing_seq = pd.DataFrame(list(set(all_seq).difference(measured_seq)), columns= ['AAs'])

# # create columns for each mutated position (missing only)
# for i in range(4):
#      missing_seq.insert(i+1, f'AA{i+1}', missing_seq['AAs'].apply(lambda x: x[i]))

# missing_seq[['fitness', 'active']] = np.nan
# missing_seq['imputed'] = True


In [10]:
# impute missing fitness values
    # average fitnes of all single mutations that are one away from missing seqence

# get list of single mutations for missing sequence
# def match_minus_one(seq, p):
#     return [sm for i, sm in enumerate(measured_seq) if i != p and sm[:p] == seq[:p] and sm[p+1:] == seq[p+1:]]

# # get subset of measured df where AAs only vary by one mutation
# def get_single_mutations(seq):
#     sm_list = []
#     for p in range(0, len(seq)):
#         sm_list.extend([sm for i, sm in enumerate(measured_seq) if i != p and sm[:p] == seq[:p] and sm[p+1:] == seq[p+1:]])
#     sm = measured[measured['AAs'].isin(sm_list)]
#     return(sm)

# # design own imputer (1.5 min runtime)
# for m in missing_seq['AAs']:
#     sm = get_single_mutations(m)
#     fit = sm['fitness'].mean()
#     missing_seq.loc[missing_seq['AAs'] == m, 'fitness'] = fit


In [9]:
# # Export to formats compatible with MLDE_lite
# missing_seq['zs score'] = np.nan
# missing_seq['active'] = missing_seq['active'].astype(bool)
# #missing_seq.rename(columns={'AAs': 'Combo', 'fitness': 'fit'}, inplace=True)
# missing_seq.to_csv('110823_TrpB_Imputed_ZS.csv')

measured.rename(columns={'AAs': 'Combo', 'fitness': 'fit'}, inplace=True)
measured['zs score'] = np.nan
measured['active'] = measured['active'].astype(bool)
measured.to_csv('120123_MLDE_tuned_realx3_mut_pred_synthetic_landscape_TrpB.csv', index = False)

# combo = pd.concat([measured, missing_seq], ignore_index= True).sort_values('AAs').reset_index(drop=True)
# combo['zs score'] = np.nan
# combo.to_csv('110823_TrpB_MeasuredAndImputed_ZS.csv', index = False)

In [7]:
measured

,AAs,sequence,AA1,AA2,AA3,AA4,num_mutations,fitness,active,imputed,zs score
0,LLVS,"[10, 10, 19, 15]",L,L,V,S,2,9.297114,True,False,NaN
1,VLVG,"[19, 10, 19, 7]",V,L,V,G,2,8.079886,True,False,NaN
2,LVVS,"[10, 19, 19, 15]",L,V,V,S,2,7.862534,True,False,NaN
3,VIVG,"[19, 9, 19, 7]",V,I,V,G,2,7.466938,True,False,NaN
4,MIVG,"[12, 9, 19, 7]",M,I,V,G,3,6.999425,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
159995,WQQM,"[17, 5, 5, 12]",W,Q,Q,M,4,-57.807239,False,False,NaN
159996,WQQY,"[17, 5, 5, 18]",W,Q,Q,Y,4,-57.807239,False,False,NaN
159997,WQQH,"[17, 5, 5, 8]",W,Q,Q,H,4,-57.807239,False,False,NaN
159998,RQQC,"[1, 5, 5, 4]",R,Q,Q,C,4,-57.807239,False,False,NaN


In [15]:
# summary stas
imputed_sum = [missing_seq['fitness'].mean(), missing_seq['fitness'].max(), missing_seq['fitness'].min()]
nonimputed_sum = [measured['fitness'].mean(), measured['fitness'].max(), measured['fitness'].min()] 

print({'imputed':imputed_sum, 'measured': nonimputed_sum})

{'imputed': [0.006061876728700011, 0.14858506827367168, -0.020904376642702862], 'measured': [0.02114863848600837, 1.0, -0.1640706614742643]}
